# Prediction Models Part 1: ElasticNet

Regarding the possible linear dependence of the features and to avoid overfitting, we apply:

1. ElasticNet regression

2. KNN

3. Neural Network models. 

In order to decide on model parameters and compare different models' accuracies, we use cross validation techniques like k-fold cross validation, LOOCV, and stratification considering the small number of observations.

In [1]:
#Importing necessary packages

import os
import numpy as np
import sklearn
import pandas as pd
import random

#For plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.manifold import SpectralEmbedding
from sklearn.manifold import TSNE

from sklearn.linear_model import ElasticNet,ElasticNetCV 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error,r2_score

from scipy import interpolate
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

path = '/Users/nesli/Desktop/SPRING2022/DSCC465/Project/'
os.chdir(path)

random.seed(265) #seed

## Read the data

We follow with the data we have from the explanatory analysis to be on the same page. Here the data excludes features that have correlation higher than $0.6$ with another feature. 

In [2]:
X=pd.read_csv('X_same_with_pcs.csv')

In [3]:
X

,Unnamed: 0,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,num_uninsured,num_primary_care_physicians,...,percentile_rank_social_vulnerability,pca_pc1,pca_pc2,pca_pc3,spec_pc1,spec_pc2,spec_pc3,tsne_pc1,tsne_pc2,tsne_pc3
0,0,0.388267,0.561029,0.342101,0.461369,0.720000,0.346707,0.015506,0.004474,0.003380,...,0.377300,-0.612896,0.063201,-0.313888,0.000252,-0.001221,0.000717,-5.478652,-3.007624,11.151594
1,1,0.285624,0.468466,0.325449,0.410596,0.800000,0.487587,0.043699,0.021010,0.020954,...,0.275700,0.993903,-0.082875,-0.187324,-0.000580,-0.000771,-0.001262,2.447136,-7.664367,-0.042642
2,2,0.653214,0.701619,0.452223,0.646799,0.560000,0.241874,0.008176,0.002321,0.001082,...,0.984700,-2.264915,0.629004,0.769147,0.001032,-0.000120,0.001038,-5.176828,8.238961,4.895467
3,3,0.344359,0.533963,0.371122,0.556291,0.780000,0.372844,0.007894,0.001871,0.001487,...,0.573700,-3.950356,0.145724,0.101242,0.001571,0.001361,-0.000986,-5.714986,16.582363,0.309691
4,4,0.414500,0.623824,0.373777,0.472406,0.840000,0.308491,0.020017,0.006939,0.001757,...,0.498600,-3.358407,-0.504476,-0.055199,0.001457,0.001013,-0.000416,-2.518751,17.053493,4.764736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,3139,0.225641,0.300716,0.317682,0.516556,0.740000,0.543080,0.005357,0.002620,0.001217,...,0.535200,1.737956,0.342858,-0.128236,-0.000870,-0.000163,-0.001173,0.091839,-1.043154,-7.836049
3140,3140,0.238373,0.304077,0.307754,0.362031,0.830000,0.418641,0.001128,0.001128,0.000541,...,0.592800,1.940699,-0.005748,0.257216,-0.000957,0.000061,-0.001021,5.933311,-3.059773,-6.624521
3141,3141,0.176860,0.298239,0.303081,0.461369,0.790000,0.474434,0.002537,0.000752,0.000406,...,0.271600,3.049711,-0.033842,0.486872,-0.001253,0.001024,0.000417,4.736609,-10.780089,-13.694742
3142,3142,0.251945,0.439812,0.359885,0.445514,0.738523,0.626147,0.184663,0.115827,0.109639,...,0.392899,0.729907,0.352062,-0.707103,-0.000480,-0.000900,-0.001095,-2.299268,-8.020679,0.646890


In [4]:
X.drop(columns=X.columns[0], axis=1, inplace=True)

In [5]:
X

,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,num_uninsured,num_primary_care_physicians,num_dentists,...,percentile_rank_social_vulnerability,pca_pc1,pca_pc2,pca_pc3,spec_pc1,spec_pc2,spec_pc3,tsne_pc1,tsne_pc2,tsne_pc3
0,0.388267,0.561029,0.342101,0.461369,0.720000,0.346707,0.015506,0.004474,0.003380,0.002061,...,0.377300,-0.612896,0.063201,-0.313888,0.000252,-0.001221,0.000717,-5.478652,-3.007624,11.151594
1,0.285624,0.468466,0.325449,0.410596,0.800000,0.487587,0.043699,0.021010,0.020954,0.012368,...,0.275700,0.993903,-0.082875,-0.187324,-0.000580,-0.000771,-0.001262,2.447136,-7.664367,-0.042642
2,0.653214,0.701619,0.452223,0.646799,0.560000,0.241874,0.008176,0.002321,0.001082,0.001031,...,0.984700,-2.264915,0.629004,0.769147,0.001032,-0.000120,0.001038,-5.176828,8.238961,4.895467
3,0.344359,0.533963,0.371122,0.556291,0.780000,0.372844,0.007894,0.001871,0.001487,0.000573,...,0.573700,-3.950356,0.145724,0.101242,0.001571,0.001361,-0.000986,-5.714986,16.582363,0.309691
4,0.414500,0.623824,0.373777,0.472406,0.840000,0.308491,0.020017,0.006939,0.001757,0.001260,...,0.498600,-3.358407,-0.504476,-0.055199,0.001457,0.001013,-0.000416,-2.518751,17.053493,4.764736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,0.225641,0.300716,0.317682,0.516556,0.740000,0.543080,0.005357,0.002620,0.001217,0.002061,...,0.535200,1.737956,0.342858,-0.128236,-0.000870,-0.000163,-0.001173,0.091839,-1.043154,-7.836049
3140,0.238373,0.304077,0.307754,0.362031,0.830000,0.418641,0.001128,0.001128,0.000541,0.000687,...,0.592800,1.940699,-0.005748,0.257216,-0.000957,0.000061,-0.001021,5.933311,-3.059773,-6.624521
3141,0.176860,0.298239,0.303081,0.461369,0.790000,0.474434,0.002537,0.000752,0.000406,0.000573,...,0.271600,3.049711,-0.033842,0.486872,-0.001253,0.001024,0.000417,4.736609,-10.780089,-13.694742
3142,0.251945,0.439812,0.359885,0.445514,0.738523,0.626147,0.184663,0.115827,0.109639,0.099175,...,0.392899,0.729907,0.352062,-0.707103,-0.000480,-0.000900,-0.001095,-2.299268,-8.020679,0.646890


In [6]:
y=pd.read_csv('y.csv')

In [7]:
y

,Unnamed: 0,life_expectancy
0,0,76.879477
1,1,78.450258
2,2,75.341935
3,3,73.571820
4,4,74.145826
...,...,...
3139,3139,79.245997
3140,3140,79.451504
3141,3141,80.549081
3142,3142,78.173254


In [8]:
y.drop(columns=y.columns[0], axis=1, inplace=True)

In [9]:
y

,life_expectancy
0,76.879477
1,78.450258
2,75.341935
3,73.571820
4,74.145826
...,...
3139,79.245997
3140,79.451504
3141,80.549081
3142,78.173254


Here, our data is not given with training and test sets. So we will be building models by dividing the data to test and train sets using different methods.

## Train-test split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=465)

## ElasticNet Regression

### See how a simple ElasticNet regression model performs on the train-test split data

We follow from [1].

In [11]:
elasticnet_model = ElasticNet().fit(X_train, y_train)

In [12]:
y_pred=elasticnet_model.predict(X_test)

In [13]:
# RMSE, root mean squared error, lower the better
np.sqrt(mean_squared_error(y_test,y_pred))

0.5232638691573344

In [14]:
# MSE
mean_squared_error(y_test,y_pred)

0.273805076765504

In [15]:
# R^2, percentage of change that our dependent variables can explain unit change on the target variable, higher the better
r2_score(y_test,y_pred)

0.9699128256469642

That is a very good score meaning that our model is able to explain the target variable by $97\%$, but we may be overfitting or the train-test split we have performed may not be representative of the data in general.

In [16]:
elasticnet_model.alpha

1.0

## This is the best model so save the outputs

These predictions are the best for when we do not use the location information. So we save them to a file.

In [17]:
y_pred=pd.DataFrame(y_pred, columns=['life_expectancy'])

In [18]:
y_pred

,life_expectancy
0,74.209070
1,78.197011
2,78.699103
3,77.823558
4,73.655822
...,...
939,74.619036
940,73.720153
941,74.293501
942,78.097144


In [19]:
y_pred.to_csv('y_pred_same_withpcs_ElasticNet.csv')

### Tuning $\lambda$

We follow from [3].

In [20]:
alphas = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1]

In [21]:
for a in alphas:
    model = ElasticNet(alpha=a).fit(X_train,y_train)   
    score = r2_score(y_test,y_pred)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)   
    print("Alpha:{0:.4f}, R2:{1:.2f}, MSE:{2:.2f}, RMSE:{3:.2f}"
       .format(a, score, mse, np.sqrt(mse)))

Alpha:0.0001, R2:0.97, MSE:0.00, RMSE:0.01
Alpha:0.0010, R2:1.00, MSE:0.00, RMSE:0.01
Alpha:0.0100, R2:1.00, MSE:0.00, RMSE:0.03
Alpha:0.1000, R2:1.00, MSE:0.01, RMSE:0.08
Alpha:0.3000, R2:1.00, MSE:0.04, RMSE:0.20
Alpha:0.5000, R2:1.00, MSE:0.09, RMSE:0.31
Alpha:0.7000, R2:0.99, MSE:0.16, RMSE:0.40
Alpha:1.0000, R2:0.98, MSE:0.27, RMSE:0.52


In [22]:
alphas = [1e-4, 1e-3, 1e-2, 1e-1]

In [23]:
for a in alphas:
    model = ElasticNet(alpha=a).fit(X_train,y_train)   
    score = r2_score(y_test,y_pred)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)   
    print("Alpha:{0:.4f}, R2:{1:.2f}, MSE:{2:.2f}, RMSE:{3:.2f}"
       .format(a, score, mse, np.sqrt(mse)))

Alpha:0.0001, R2:0.97, MSE:0.00, RMSE:0.01
Alpha:0.0010, R2:1.00, MSE:0.00, RMSE:0.01
Alpha:0.0100, R2:1.00, MSE:0.00, RMSE:0.03
Alpha:0.1000, R2:1.00, MSE:0.01, RMSE:0.08


$R^2$ being $1$ means that the model overfits. $R^2$ is the minimum when $\alpha=0$ and maximum when $\alpha=1$. This means that the model wants us to penalize each term to the maximum as the mathematical form of ElasticNet is an optimization problem given as: [2]

$$J(W)=\frac{1}{2N}\sum_{i=1}^{N}(W_{0}+W_{1}X_{1}^{i}+...+W_{p}X_{p}^{i}-Y_{i})^{2}+\frac{\lambda_{1}}{2N}\sum_{j=1}^{p}|W_{j}|+\frac{\lambda_{2}}{2N}\sum_{j=1}^{p}W_{j}^{2}$$

Here, $J$ is the cost function, $N$ is the number of observations, $p$ is the number of features, $W_{i}$ are weights, $X_{j}^{i}$ are $j$th feature's $i$th observation, $Y_{i}$ is the target variable, and $lambda_{1}$ and $lambda_{2}$ are the penalizing coefficients for $L1$ and $L2$ regularization terms.

We are alredy performing regularization on our data by choosing to use ElasticNet regression to penalize the model complexity and to avoid overfitting. But we still have an overfitting problem. 

To avoid overfitting, we can try other cross validation techniques like LOOCV, k-fold, and stratification.

### 10-fold cross validation

In [24]:
elastic_cv=ElasticNetCV(cv=10)
model = elastic_cv.fit(X, y)

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
model.alpha_

0.05052024529986243

In [26]:
#Test on the whole data
y_pred = model.predict(X)

In [27]:
score = r2_score(y,y_pred)
mse = mean_squared_error(y, y_pred)
print("R2:{0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}".format(score, mse, np.sqrt(mse)))

R2:1.000, MSE:0.00, RMSE:0.05


### LOOCV

In [28]:
elastic_cv=ElasticNetCV(cv=len(X))
model = elastic_cv.fit(X, y)

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
model.alpha_

0.05052024529986243

In [30]:
#Test on the whole data
y_pred = model.predict(X)

In [31]:
score = r2_score(y,y_pred)
mse = mean_squared_error(y, y_pred)
print("R2:{0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}".format(score, mse, np.sqrt(mse)))

R2:1.000, MSE:0.00, RMSE:0.05


### Stratification

We apply stratification based on one of the important features found: 'percent_adults_with_obesity'.

In [32]:
X_stratified_test=X.groupby('percent_adults_with_obesity').apply(lambda x: x.sample(frac=.3))

In [33]:
df_test = X_stratified_test.droplevel(level=0)

In [34]:
df_test

,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,num_uninsured,num_primary_care_physicians,num_dentists,...,percentile_rank_social_vulnerability,pca_pc1,pca_pc2,pca_pc3,spec_pc1,spec_pc2,spec_pc3,tsne_pc1,tsne_pc2,tsne_pc3
2798,0.080886,0.260270,0.075254,0.048565,0.83,0.394229,0.006484,0.003856,0.007030,0.002405,...,0.0694,6.578498,0.181859,-0.635024,-0.001415,0.001637,0.001632,-2.992976,-13.752817,-21.892880
298,0.086381,0.300407,0.237363,0.070640,0.81,0.724502,0.004793,0.001961,0.004191,0.002863,...,0.0528,5.916509,0.188783,-0.217232,-0.001415,0.001636,0.001629,-2.537816,-13.519159,-20.296703
206,0.074721,0.242053,0.107258,0.092715,0.88,0.600002,0.016070,0.011917,0.052319,0.035044,...,0.2623,7.919433,0.465265,0.141876,-0.001416,0.001638,0.001634,-3.258019,-15.951128,-22.079275
2980,0.151646,0.391600,0.171204,0.130243,0.87,0.500195,0.000846,0.001081,0.001757,0.001374,...,0.1092,8.024713,-0.390313,0.634425,-0.001415,0.001637,0.001630,-2.417901,-16.235891,-22.375538
278,0.130890,0.304296,0.249272,0.132450,0.84,0.595238,0.011277,0.005648,0.008517,0.005726,...,0.1207,4.694195,0.313469,0.125144,-0.001403,0.001589,0.001526,-1.395549,-11.691757,-17.099138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0.522458,0.601172,0.400105,0.746137,0.58,0.193973,0.001128,0.000568,0.000270,0.002320,...,0.8039,-2.121054,-0.136940,0.722389,0.000975,-0.000257,0.001172,-1.532239,9.283689,8.884869
1087,0.562595,0.768072,0.568584,0.748344,0.69,0.292152,0.001973,0.000206,0.003437,0.000115,...,0.8243,-8.534478,-0.275691,0.002508,0.001629,0.001548,-0.001319,-1.084326,19.281898,-11.310092
1134,0.437050,0.599683,0.467384,0.772627,0.73,0.590545,0.007330,0.001644,0.000270,0.000344,...,0.6816,-2.048130,0.138399,0.337875,0.000957,-0.000314,0.001263,-4.394534,9.204858,9.988652
2131,0.621356,0.792322,0.613662,0.779249,0.68,0.190718,0.005075,0.004893,0.001082,0.000687,...,0.8723,-4.384650,0.261623,0.063708,0.001601,0.001458,-0.001155,-2.830037,14.451518,-2.452297


In [35]:
df_train = X[~X.index.isin(df_test.index)]

In [36]:
df_train

,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,num_uninsured,num_primary_care_physicians,num_dentists,...,percentile_rank_social_vulnerability,pca_pc1,pca_pc2,pca_pc3,spec_pc1,spec_pc2,spec_pc3,tsne_pc1,tsne_pc2,tsne_pc3
0,0.388267,0.561029,0.342101,0.461369,0.720000,0.346707,0.015506,0.004474,0.003380,0.002061,...,0.377300,-0.612896,0.063201,-0.313888,0.000252,-0.001221,0.000717,-5.478652,-3.007624,11.151594
1,0.285624,0.468466,0.325449,0.410596,0.800000,0.487587,0.043699,0.021010,0.020954,0.012368,...,0.275700,0.993903,-0.082875,-0.187324,-0.000580,-0.000771,-0.001262,2.447136,-7.664367,-0.042642
3,0.344359,0.533963,0.371122,0.556291,0.780000,0.372844,0.007894,0.001871,0.001487,0.000573,...,0.573700,-3.950356,0.145724,0.101242,0.001571,0.001361,-0.000986,-5.714986,16.582363,0.309691
5,0.696028,0.623796,0.477367,0.547461,0.430000,0.205844,0.006766,0.000854,0.000406,0.000229,...,0.936300,-4.073538,0.530011,0.613049,0.001581,0.001394,-0.001045,-6.627830,14.739326,-1.998529
6,0.602069,0.742656,0.445628,0.682119,0.660000,0.194472,0.014096,0.001906,0.000676,0.000687,...,0.916600,-4.652710,0.117852,0.253996,0.001610,0.001485,-0.001204,-2.642695,15.719512,-3.262039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,0.225641,0.300716,0.317682,0.516556,0.740000,0.543080,0.005357,0.002620,0.001217,0.002061,...,0.535200,1.737956,0.342858,-0.128236,-0.000870,-0.000163,-0.001173,0.091839,-1.043154,-7.836049
3140,0.238373,0.304077,0.307754,0.362031,0.830000,0.418641,0.001128,0.001128,0.000541,0.000687,...,0.592800,1.940699,-0.005748,0.257216,-0.000957,0.000061,-0.001021,5.933311,-3.059773,-6.624521
3141,0.176860,0.298239,0.303081,0.461369,0.790000,0.474434,0.002537,0.000752,0.000406,0.000573,...,0.271600,3.049711,-0.033842,0.486872,-0.001253,0.001024,0.000417,4.736609,-10.780089,-13.694742
3142,0.251945,0.439812,0.359885,0.445514,0.738523,0.626147,0.184663,0.115827,0.109639,0.099175,...,0.392899,0.729907,0.352062,-0.707103,-0.000480,-0.000900,-0.001095,-2.299268,-8.020679,0.646890


In [37]:
df_y_train=pd.DataFrame(y['life_expectancy'][df_train.index])

In [38]:
df_y_train

,life_expectancy
0,76.879477
1,78.450258
3,73.571820
5,73.530895
6,72.918309
...,...
3139,79.245997
3140,79.451504
3141,80.549081
3142,78.173254


In [39]:
df_y_test=pd.DataFrame(y['life_expectancy'][df_test.index])

In [40]:
elastic_cv=ElasticNetCV()
model = elastic_cv.fit(df_train, df_y_train)

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [41]:
model.alpha_

0.051100372022482576

In [42]:
#Test on the whole data
y_pred = model.predict(df_test)

In [43]:
score = r2_score(df_y_test,y_pred)
mse = mean_squared_error(df_y_test, y_pred)
print("R2:{0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}".format(score, mse, np.sqrt(mse)))

R2:1.000, MSE:0.00, RMSE:0.05


The models we obtain from ElasticNet are overfitting. One way to solve that could be to remove the columns that contain the principal components returned by different dimensionality reduction methods.

## Remove PCA, spectral embedding, and t-SNE columns

In [44]:
X

,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,num_uninsured,num_primary_care_physicians,num_dentists,...,percentile_rank_social_vulnerability,pca_pc1,pca_pc2,pca_pc3,spec_pc1,spec_pc2,spec_pc3,tsne_pc1,tsne_pc2,tsne_pc3
0,0.388267,0.561029,0.342101,0.461369,0.720000,0.346707,0.015506,0.004474,0.003380,0.002061,...,0.377300,-0.612896,0.063201,-0.313888,0.000252,-0.001221,0.000717,-5.478652,-3.007624,11.151594
1,0.285624,0.468466,0.325449,0.410596,0.800000,0.487587,0.043699,0.021010,0.020954,0.012368,...,0.275700,0.993903,-0.082875,-0.187324,-0.000580,-0.000771,-0.001262,2.447136,-7.664367,-0.042642
2,0.653214,0.701619,0.452223,0.646799,0.560000,0.241874,0.008176,0.002321,0.001082,0.001031,...,0.984700,-2.264915,0.629004,0.769147,0.001032,-0.000120,0.001038,-5.176828,8.238961,4.895467
3,0.344359,0.533963,0.371122,0.556291,0.780000,0.372844,0.007894,0.001871,0.001487,0.000573,...,0.573700,-3.950356,0.145724,0.101242,0.001571,0.001361,-0.000986,-5.714986,16.582363,0.309691
4,0.414500,0.623824,0.373777,0.472406,0.840000,0.308491,0.020017,0.006939,0.001757,0.001260,...,0.498600,-3.358407,-0.504476,-0.055199,0.001457,0.001013,-0.000416,-2.518751,17.053493,4.764736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,0.225641,0.300716,0.317682,0.516556,0.740000,0.543080,0.005357,0.002620,0.001217,0.002061,...,0.535200,1.737956,0.342858,-0.128236,-0.000870,-0.000163,-0.001173,0.091839,-1.043154,-7.836049
3140,0.238373,0.304077,0.307754,0.362031,0.830000,0.418641,0.001128,0.001128,0.000541,0.000687,...,0.592800,1.940699,-0.005748,0.257216,-0.000957,0.000061,-0.001021,5.933311,-3.059773,-6.624521
3141,0.176860,0.298239,0.303081,0.461369,0.790000,0.474434,0.002537,0.000752,0.000406,0.000573,...,0.271600,3.049711,-0.033842,0.486872,-0.001253,0.001024,0.000417,4.736609,-10.780089,-13.694742
3142,0.251945,0.439812,0.359885,0.445514,0.738523,0.626147,0.184663,0.115827,0.109639,0.099175,...,0.392899,0.729907,0.352062,-0.707103,-0.000480,-0.000900,-0.001095,-2.299268,-8.020679,0.646890


In [45]:
# Select the last 9 columns [5]
pcs = X.T.tail(9).T

In [46]:
pcs

,pca_pc1,pca_pc2,pca_pc3,spec_pc1,spec_pc2,spec_pc3,tsne_pc1,tsne_pc2,tsne_pc3
0,-0.612896,0.063201,-0.313888,0.000252,-0.001221,0.000717,-5.478652,-3.007624,11.151594
1,0.993903,-0.082875,-0.187324,-0.000580,-0.000771,-0.001262,2.447136,-7.664367,-0.042642
2,-2.264915,0.629004,0.769147,0.001032,-0.000120,0.001038,-5.176828,8.238961,4.895467
3,-3.950356,0.145724,0.101242,0.001571,0.001361,-0.000986,-5.714986,16.582363,0.309691
4,-3.358407,-0.504476,-0.055199,0.001457,0.001013,-0.000416,-2.518751,17.053493,4.764736
...,...,...,...,...,...,...,...,...,...
3139,1.737956,0.342858,-0.128236,-0.000870,-0.000163,-0.001173,0.091839,-1.043154,-7.836049
3140,1.940699,-0.005748,0.257216,-0.000957,0.000061,-0.001021,5.933311,-3.059773,-6.624521
3141,3.049711,-0.033842,0.486872,-0.001253,0.001024,0.000417,4.736609,-10.780089,-13.694742
3142,0.729907,0.352062,-0.707103,-0.000480,-0.000900,-0.001095,-2.299268,-8.020679,0.646890


In [47]:
X_removed = X.iloc[: , :-9]

In [48]:
X_removed

,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,num_uninsured,num_primary_care_physicians,num_dentists,...,num_minorities,num_institutionalized_in_group_quarters,percentile_rank_age_65_and_older,percentile_rank_age_17_and_younger,percentile_rank_minorities,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,0.388267,0.561029,0.342101,0.461369,0.720000,0.346707,0.015506,0.004474,0.003380,0.002061,...,0.001816,0.002771,0.196400,0.831300,0.633900,0.679100,0.726800,0.125100,0.288100,0.377300
1,0.285624,0.468466,0.325449,0.410596,0.800000,0.487587,0.043699,0.021010,0.020954,0.012368,...,0.004554,0.016464,0.643700,0.447600,0.525300,0.973300,0.538700,0.343800,0.332400,0.275700
2,0.653214,0.701619,0.452223,0.646799,0.560000,0.241874,0.008176,0.002321,0.001082,0.001031,...,0.001954,0.016583,0.417400,0.361700,0.904200,0.281400,0.937000,0.942700,0.931200,0.984700
3,0.344359,0.533963,0.371122,0.556291,0.780000,0.372844,0.007894,0.001871,0.001487,0.000573,...,0.000773,0.011317,0.258500,0.308800,0.645000,0.407200,0.924900,0.915600,0.666300,0.573700
4,0.414500,0.623824,0.373777,0.472406,0.840000,0.308491,0.020017,0.006939,0.001757,0.001260,...,0.000966,0.003122,0.490900,0.646600,0.423800,0.134400,0.846500,0.151500,0.182700,0.498600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,0.225641,0.300716,0.317682,0.516556,0.740000,0.543080,0.005357,0.002620,0.001217,0.002061,...,0.000348,0.001380,0.044600,0.966900,0.421200,0.770500,0.875800,0.215900,0.656200,0.535200
3140,0.238373,0.304077,0.307754,0.362031,0.830000,0.418641,0.001128,0.001128,0.000541,0.000687,...,0.000199,0.000995,0.730000,0.800700,0.536100,0.346400,0.511600,0.537400,0.514200,0.592800
3141,0.176860,0.298239,0.303081,0.461369,0.790000,0.474434,0.002537,0.000752,0.000406,0.000573,...,0.000076,0.002013,0.615100,0.307500,0.288800,0.552100,0.861800,0.807700,0.633900,0.271600
3142,0.251945,0.439812,0.359885,0.445514,0.738523,0.626147,0.184663,0.115827,0.109639,0.099175,...,0.043998,0.091127,0.166922,0.744178,0.675575,0.879288,0.055045,0.329812,0.384675,0.392899


## Now, repeat without the principal components 

## Train-test split

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_removed, y, test_size=0.3, random_state=465)

## See how a simple ElasticNet regression model performs on the train-test split data

We follow from [1].

In [50]:
elasticnet_model = ElasticNet().fit(X_train, y_train)

In [51]:
y_pred=elasticnet_model.predict(X_test)

In [52]:
# RMSE, root mean squared error, lower the better
np.sqrt(mean_squared_error(y_test,y_pred))

3.01676996751846

In [53]:
mean_squared_error(y_test,y_pred)

9.10090103692133

In [54]:
# R^2, percentage of change that our dependent variables can explain unit change on the target variable, higher the better
r2_score(y_test,y_pred)

-5.5950394539650006e-05

In [55]:
elasticnet_model.alpha

1.0

### Tuning $\lambda$

We follow from [3].

In [56]:
alphas = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1]

In [57]:
for a in alphas:
    model = ElasticNet(alpha=a).fit(X_train,y_train)   
    score = r2_score(y_test,y_pred)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)   
    print("Alpha:{0:.4f}, R2:{1:.2f}, MSE:{2:.2f}, RMSE:{3:.2f}"
       .format(a, score, mse, np.sqrt(mse)))

Alpha:0.0001, R2:-0.00, MSE:2.87, RMSE:1.69
Alpha:0.0010, R2:0.68, MSE:2.94, RMSE:1.71
Alpha:0.0100, R2:0.68, MSE:3.27, RMSE:1.81
Alpha:0.1000, R2:0.64, MSE:4.82, RMSE:2.20
Alpha:0.3000, R2:0.47, MSE:6.86, RMSE:2.62
Alpha:0.5000, R2:0.25, MSE:8.12, RMSE:2.85
Alpha:0.7000, R2:0.11, MSE:8.71, RMSE:2.95
Alpha:1.0000, R2:0.04, MSE:9.10, RMSE:3.02


This shows that the model was overfitting earlier because of the added columns from the dimensionality reduction and ElasticNet, although it is more effective in deciding on the features for the model more than Lasso or Ridge, still could not handle that issue well enough.

In [58]:
alphas = [1e-3, 5e-3, 1e-2]

In [59]:
for a in alphas:
    model = ElasticNet(alpha=a).fit(X_train,y_train)   
    score = r2_score(y_test,y_pred)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)   
    print("Alpha:{0:.4f}, R2:{1:.2f}, MSE:{2:.2f}, RMSE:{3:.2f}"
       .format(a, score, mse, np.sqrt(mse)))

Alpha:0.0010, R2:-0.00, MSE:2.94, RMSE:1.71
Alpha:0.0050, R2:0.68, MSE:3.13, RMSE:1.77
Alpha:0.0100, R2:0.66, MSE:3.27, RMSE:1.81


We should choose $\alpha=0.005$.

### 10-fold cross validation

In [60]:
elastic_cv=ElasticNetCV(cv=10)
model = elastic_cv.fit(X_removed, y)

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [61]:
#Test on the whole data
y_pred = model.predict(X_removed)

In [62]:
score = r2_score(y,y_pred)
mse = mean_squared_error(y, y_pred)
print("R2:{0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}".format(score, mse, np.sqrt(mse)))

R2:0.701, MSE:2.71, RMSE:1.65


Having 10-fold cross validation showed that train-test split actually have obtained better predictions.

In [63]:
k_list=[2, 10, 100, 1000, len(X_removed)]

In [64]:
for k in k_list:
    elastic_cv=ElasticNetCV(cv=k)
    model = elastic_cv.fit(X_removed, y)
    y_pred = model.predict(X_removed)
    score = r2_score(y,y_pred)
    mse = mean_squared_error(y, y_pred)
    print("k:{0:.3f},R2:{1:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}".format(k,score, mse, np.sqrt(mse)))

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


k:2.000,R2:0.701, MSE:0.70, RMSE:2.72
k:10.000,R2:0.701, MSE:0.70, RMSE:2.71


/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


k:100.000,R2:0.707, MSE:0.71, RMSE:2.66


/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


k:1000.000,R2:0.707, MSE:0.71, RMSE:2.66


/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


k:3144.000,R2:0.707, MSE:0.71, RMSE:2.66


### LOOCV

In [65]:
elastic_cv=ElasticNetCV(cv=len(X_removed))
model = elastic_cv.fit(X_removed, y)

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [66]:
model.alpha_

0.0009944333946377431

In [67]:
#Test on the whole data
y_pred = model.predict(X_removed)

In [68]:
score = r2_score(y,y_pred)
mse = mean_squared_error(y, y_pred)
print("R2:{0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}".format(score, mse, np.sqrt(mse)))

R2:0.707, MSE:2.66, RMSE:1.63


### Stratification

We follow from [4].

In [69]:
X_stratified_test=X_removed.groupby('percent_adults_with_obesity').apply(lambda x: x.sample(frac=.3))

In [70]:
df_test = X_stratified_test.droplevel(level=0)

In [71]:
df_test

,percent_fair_or_poor_health,average_number_of_mentally_unhealthy_days,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_excessive_drinking,num_driving_deaths,num_uninsured,num_primary_care_physicians,num_dentists,...,num_minorities,num_institutionalized_in_group_quarters,percentile_rank_age_65_and_older,percentile_rank_age_17_and_younger,percentile_rank_minorities,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
1858,0.227856,0.380447,0.182941,0.048565,0.83,0.816908,0.050465,0.091673,0.298635,0.332913,...,0.117431,0.341054,0.2229,0.0105,0.8968,1.0000,0.0032,0.7393,0.9682,0.5244
252,0.142493,0.321376,0.213030,0.070640,0.86,0.516412,0.005075,0.001471,0.002433,0.001374,...,0.000318,0.006583,0.8819,0.0216,0.4320,0.2942,0.4117,0.8574,0.2174,0.1605
260,0.211871,0.281568,0.246069,0.092715,0.81,0.903223,0.072174,0.071913,0.126132,0.061040,...,0.041978,0.085545,0.0493,0.2700,0.8634,0.9959,0.0108,0.5661,0.8774,0.4798
269,0.113050,0.293242,0.232134,0.130243,0.85,0.553044,0.003947,0.001453,0.001622,0.000687,...,0.000248,0.001708,0.2311,0.0783,0.4304,0.9589,0.2076,0.5323,0.5307,0.1032
278,0.130890,0.304296,0.249272,0.132450,0.84,0.595238,0.011277,0.005648,0.008517,0.005726,...,0.001510,0.010718,0.2022,0.1570,0.5798,0.8424,0.6138,0.7224,0.6055,0.1207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2708,0.433549,0.307665,0.222913,0.746137,0.86,0.549452,0.018607,0.002881,0.000946,0.000458,...,0.001581,0.011832,0.1067,0.7663,0.9761,0.7287,0.7335,0.9615,0.9768,0.7529
1087,0.562595,0.768072,0.568584,0.748344,0.69,0.292152,0.001973,0.000206,0.003437,0.000115,...,0.000037,0.000441,0.3999,0.3448,0.2143,0.0993,0.9004,0.4311,0.7679,0.8243
1134,0.437050,0.599683,0.467384,0.772627,0.73,0.590545,0.007330,0.001644,0.000270,0.000344,...,0.000709,0.019643,0.1885,0.4161,0.6218,0.0315,0.9344,0.9736,0.4620,0.6816
65,0.648652,0.707224,0.454096,0.779249,0.25,0.153226,0.006766,0.000954,0.000541,0.000115,...,0.001095,0.001465,0.5524,0.6192,0.9742,0.0926,0.9819,0.5750,0.5798,0.8711


In [72]:
df_train = X_removed[~X_removed.index.isin(df_test.index)]

In [73]:
df_y_train=pd.DataFrame(y['life_expectancy'][df_train.index])

In [74]:
df_y_test=pd.DataFrame(y['life_expectancy'][df_test.index])

In [75]:
elastic_cv=ElasticNetCV()
model = elastic_cv.fit(df_train, df_y_train)

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [76]:
#Test on the whole data
y_pred = model.predict(df_test)

In [77]:
score = r2_score(df_y_test,y_pred)
mse = mean_squared_error(df_y_test, y_pred)
print("R2:{0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}".format(score, mse, np.sqrt(mse)))

R2:0.694, MSE:2.57, RMSE:1.60


We see that stratified data with k-fold cross validation returns the most reasonable model with the highest $R^{2}$ value that is not overfitting by approaching 1. But we can still change $k$ to find the best value for it.

## References

[1] https://medium.com/mlearning-ai/elasticnet-regression-fundamentals-and-modeling-in-python-8668f3c2e39e

[2] Caliskan, Cantay. DSCC 465: Introduction to Statistical Machine Learning, Spring 2022, University of Rochester, Rochester NY.

[3] https://www.datatechnotes.com/2019/08/elasticnet-regression-example-in-python.html

[4] https://stackoverflow.com/questions/41035187/stratified-samples-from-pandas

[5] https://thispointer.com/pandas-select-last-n-columns-of-dataframe/

[6] https://www.geeksforgeeks.org/data-normalization-with-pandas/